In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [62]:
def scrap_macrotrend(link, variable_name=None, company_name=None):
    url = link
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    if variable_name == "current_ratio":
        tb = soup.find_all('table')[0]
        tb = tb.find_all("td")
        variable = []
        for i in range(0, len(tb), 4):
            variable.append(tb[i:(i+4)])
        date = []
        current_assets = []
        current_liabilities = []
        current_ratio = []
        for fila in variable:
            date.append(fila[0].text)
            current_assets.append(fila[1].text)
            current_liabilities.append(fila[2].text)
            current_ratio.append(fila[3].text)
            
        df = pd.DataFrame.from_dict({"date":date, "current_assets":current_assets,
                                    "current_liabilities":current_liabilities,
                                    "current_ratio":current_ratio})
            
    else:
        tb = soup.find_all('table', class_='historical_data_table table')
        tb = tb[1]
        tb = tb.find_all("td")    
        date = []
        variable = []
        for i in range(len(tb)):  
            if i % 2 == 0:
                date.append(tb[i].text)
            else:
                variable.append(tb[i].text) 
        df = pd.DataFrame.from_dict({"date":date, variable_name:variable})
    return df

In [64]:
apple_rd = scrap_macrotrend('https://www.macrotrends.net/stocks/charts/AAPL/apple/research-development-expenses',
                "r&d", "Apple")
apple_asset = scrap_macrotrend('https://www.macrotrends.net/stocks/charts/AAPL/apple/total-assets',
                              "assets", "Apple")
apple_liabilities = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/AAPL/apple/total-liabilities",
                                    "liabilities", "Apple")
apple_ratio = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/AAPL/apple/current-ratio",
                "current_ratio", "Apple")
apple_ratio.head(2)

,date,current_assets,current_liabilities,current_ratio
0,2020-09-30,$143.71B,$105.39B,1.36
1,2020-06-30,$140.07B,$95.32B,1.47


In [68]:
apple = apple_rd.merge(apple_asset, on=["date"], how="left")
apple = apple.merge(apple_liabilities, on=["date"], how="left")
apple = apple.merge(apple_ratio, on=["date"], how="left")
apple["r&d"] = apple["r&d"].str.replace(',', '').str.replace('$', '').astype(float)
apple["assets"] = apple["assets"].str.replace(',', '').str.replace('$', '').astype(float)
apple["liabilities"] = apple["liabilities"].str.replace(',', '').str.replace('$', '').astype(float)
apple["current_assets"] = apple["current_assets"].str.replace(',', '').str.replace('$', '').str.replace("B", "").astype(float)
apple["current_liabilities"] = apple["current_liabilities"].str.replace(',', '').str.replace('$', '').str.replace("B", "").astype(float)
#apple["liquidity"] =  apple["assets"] / apple["liabilities"]
apple.to_csv("output/apple_data.csv", index=None)
apple.head()

,date,r&d,assets,liabilities,current_assets,current_liabilities,current_ratio
0,2020-09-30,4978.0,323888.0,258549.0,143.71,105.39,1.36
1,2020-06-30,4758.0,317344.0,245062.0,140.07,95.32,1.47
2,2020-03-31,4565.0,320400.0,241975.0,143.75,96.09,1.50
3,2019-12-31,4451.0,340618.0,251087.0,163.23,102.16,1.60
4,2019-09-30,4110.0,338516.0,248028.0,162.82,105.72,1.54


In [69]:
google_rd = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/GOOG/alphabet/research-development-expenses",
                            "r&d", "Alphabet")
google_asset = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/GOOG/alphabet/total-assets",
                               "assets", "Alphabet")
google_liabilities = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/GOOG/alphabet/total-liabilities",
                                      "liabilities", "Alphabet")
google_ratio = scrap_macrotrend("https://www.macrotrends.net/stocks/charts/GOOG/alphabet/current-ratio",
                               "current_ratio", "Alphabet")
google_ratio.head(2)

,date,current_assets,current_liabilities,current_ratio
0,2020-09-30,$164.37B,$48.20B,3.41
1,2020-06-30,$149.07B,$43.66B,3.41


In [72]:
google = google_rd.merge(google_asset, on=["date"], how="left")
google = google.merge(google_liabilities, on=["date"], how="left")
google = google.merge(google_ratio, on=["date"], how="left")
google["r&d"] = google["r&d"].str.replace(',', '').str.replace('$', '').astype(float)
google["assets"] = google["assets"].str.replace(',', '').str.replace('$', '').astype(float)
google["liabilities"] = google["liabilities"].str.replace(',', '').str.replace('$', '').astype(float)
google["current_assets"] = google["current_assets"].str.replace(',', '').str.replace('$', '').str.replace("B", "").astype(float)
google["current_liabilities"] = google["current_liabilities"].str.replace(',', '').str.replace('$', '').str.replace("B", "").astype(float)
#google["liquidity"] = google["assets"] / google["liabilities"]
google.to_csv("output/google_data.csv", index=None)
google.head()

,date,r&d,assets,liabilities,current_assets,current_liabilities,current_ratio
0,2020-09-30,6856.0,299243.0,86323.0,164.37,48.20,3.41
1,2020-06-30,6875.0,278492.0,71170.0,149.07,43.66,3.41
2,2020-03-31,6820.0,273403.0,69744.0,147.02,40.19,3.66
3,2019-12-31,7222.0,275909.0,74467.0,152.58,45.22,3.37
4,2019-09-30,6554.0,263044.0,68075.0,148.36,39.22,3.78
